# Day 10: Pipe Maze

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

- `|` is a vertical pipe connecting north and south.
- `-` is a horizontal pipe connecting east and west.
- `L` is a 90-degree bend connecting north and east.
- `J` is a 90-degree bend connecting north and west.
- `7` is a 90-degree bend connecting south and west.
- `F` is a 90-degree bend connecting south and east.
- `.` is ground; there is no pipe in this tile.
- `S` is the starting position of the animal; there is a pipe on this - tile, but your sketch doesn't show what shape the pipe has.
Based on the acoustics of the animal's scurrying, you're confident - the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

```
.....
.F-7.
.|.|.
.L-J.
.....
```

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

```
-L|F7
7S-7|
L|7||
-L-J|
L|-JF
```

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

```
7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
```

If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

```
.....
.S-7.
.|.|.
.L-J.
.....
```

You can count the distance each tile in the loop is from the starting point like this:

```
.....
.012.
.1.3.
.234.
.....
```

In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

```
..F7.
.FJ|.
SJ.L7
|F--J
LJ...
```

Here are the distances for each tile on that loop:

```
..45.
.236.
01.78
14567
23...
```

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [13]:
from itertools import product
import numpy as np

grid = []
with open("2023_day_10_input.txt") as f:
    for line in f:
        grid.append([s for s in line if s!='\n'])

i_max = len(grid)
j_max = len(grid[0])

start_pos = None

for i, j in product(range(i_max), range(j_max)):
    if 'S' == grid[i][j]:
        start_pos = (i, j)
        break

distance = {}
pos_list = [start_pos]
loop = [start_pos]
step = 0
while len(pos_list) > 0:
    new_pos_list = []
    step += 1
    for pos in pos_list:
        symbol = grid[pos[0]][pos[1]]
        for di, dj in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            new_pos = (pos[0] + di, pos[1] + dj)
            if new_pos[0] < 0 or new_pos[0] >= i_max or new_pos[1] < 0 or new_pos[1] >= j_max:
                continue
            new_symbol = grid[new_pos[0]][new_pos[1]]
            keep = False
            if di == -1 and new_symbol in ['|', '7', 'F'] and symbol in ['|', 'L', 'J', 'S']:
                keep = True
            if di == 1 and new_symbol in ['|', 'L', 'J'] and symbol in ['|', '7', 'F', 'S']:
                keep = True
            if dj == -1 and new_symbol in ['-', 'F', 'L'] and symbol in ['-', '7', 'J', 'S']:
                keep = True
            if dj == 1 and new_symbol in ['-', '7', 'J'] and symbol in ['-', 'F', 'L', 'S']:
                keep = True
            if keep and new_pos not in distance.keys() and new_pos not in new_pos_list:
                new_pos_list.append(new_pos)
    pos_list = new_pos_list
    if len(pos_list) == 2:
        loop = [pos_list[0], *loop, pos_list[-1]]
    elif len(pos_list) == 1:
        loop = [*loop, pos_list[0]]
    for pos in pos_list:
        distance[pos] = step

max(distance.values())

6903

# Part Two

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

```
...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........
```

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

```
...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....
```

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```

In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

```
.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
```

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

```
OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO
```

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

Here are just the tiles that are enclosed by the loop marked with I:

```
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
```

In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?



In [21]:
ground = {}
zones = {}
zone_counter = 0
for i, j in product(range(i_max), range(j_max)):
    if (i, j) in loop:
        continue
    found_zones_list = []
    for di, dj in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
        new_pos = (i + di, j + dj)
        if new_pos in ground.keys():
            found_zones_list.append(ground[new_pos])
    found_zones = set(found_zones_list)
    if len(found_zones) == 0:
        zone_counter += 1
        zone = zone_counter
        ground[(i,j)] = zone
        zones[zone] = [(i, j)]
    elif len(found_zones) == 1:
        zone = found_zones.pop()
        ground[(i, j)] = zone
        zones[zone].append((i, j))
    else:
        zone_counter += 1
        new_zone = zone_counter
        zones[new_zone] = [(i, j)]
        for zone in found_zones:
            zones[new_zone].extend(zones[zone])
            del zones[zone]
        for pos in zones[new_zone]:
            ground[pos] = new_zone
       

In [22]:
from typing import Dict, Set, Tuple

borders: Dict[int,Set[Tuple[int,int]]] = {}

for zone_id in zones.keys():
    border = set()
    enclosed = True
    for pos in zones[zone_id]:
        for di, dj in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            new_pos = (pos[0] + di, pos[1] + dj)
            if new_pos[0] < 0 or new_pos[0] >= i_max or new_pos[1] < 0 or new_pos[1] >= j_max:
                enclosed = False
                break
            if new_pos not in loop and new_pos not in zones[zone_id]:
                enclosed = False
                break
            if new_pos in zones[zone_id]:
                continue
            border.add(new_pos)
        if not enclosed:
            break
    if enclosed:
        borders[zone_id] = border

In [17]:
i = loop.index(start_pos)
i1, j1 = loop[i - 1]
i2, j2 = start_pos
i3, j3 = loop[i + 1]

S = "S"

if j2 == j1 + 1:
    if j3 == j2 + 1:
        S = "-"
    elif i3 == i2 + 1:
        S = "7"
    if i3 == i2 - 1:
        S = "J"
elif j2 == j1 - 1:
    if j3 == j2 - 1:
        S = "-"
    elif i3 == i2 + 1:
        S = "F"
    if i3 == i2 - 1:
        S = "L"
elif i2 == i1 + 1:
    if i3 == i2 + 1:
        S = "|"
    elif j3 == j2 + 1:
        S = "L"
    if j3 == j2 - 1:
        S = "J"
elif i2 == i1 - 1:
    if i3 == i2 - 1:
        S = "|"
    elif j3 == j2 + 1:
        S = "F"
    if j3 == j2 - 1:
        S = "J"

grid[start_pos[0]][start_pos[1]] = S

In [18]:
from functools import lru_cache

@lru_cache
def turn_score(i1, j1, i2, j2):
    if j2 == j1 + 1:
        if grid[i2][j2] == "7":
            return 1
        elif grid[i2][j2] == "J":
            return -1
    elif j2 == j1 - 1:
        if grid[i2][j2] == "L":
            return 1
        elif grid[i2][j2] == "F":
            return -1
    elif i2 == i1 + 1:
        if grid[i2][j2] == "J":
            return 1
        elif grid[i2][j2] == "L":
            return -1
    elif i2 == i1 - 1:
        if grid[i2][j2] == "F":
            return 1
        elif grid[i2][j2] == "7":
            return -1
    return 0

In [20]:
res = 0

for zone_id in borders.keys():
    border = list(borders[zone_id])
    pos = border[0]
    i = loop.index(pos)
    step = 0
    inside_loop = True
    for (i1, j1), (i2, j2) in pairwise(loop[i:] + loop[:i + 1]):
        step += turn_score(i1, j1, i2, j2)
        if (i2, j2) in border:
            if abs(step) > 2:
                inside_loop = False
                break
            step = 0
    if inside_loop:
        res += len(zones[zone_id])
res

265